---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

df.head(10)



0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    month_name = '(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?),?\.?'
    month_num = '(?:[1-9]|0[1-9]|1[0-2])'
    day_num = '(?:0?[1-9]|1[0-9]|2[0-9]|3[0-1])(?:st|nd|th)?,?' 

    month_exp = fr'(?P<month>{month_num}|{month_name})' 
    day_exp = fr'(?P<day>{day_num})'
    year_exp = '(?P<year>\d{4}|\d{2})'
    year_4num = '(?P<year>\d{4})'
    sep_exp = '(?P<sep>\/|-|\s)'

    exp_1 = fr'(?P<date>{month_exp}(?P<sep>\/|-|\s){day_exp}(?P=sep){year_exp})'
    exp_2 = fr'(?P<date>{day_exp}(?P<sep>\/|-|\s){month_exp}(?P=sep){year_exp})'
    exp_3 = fr'(?P<date>{month_name}\s{year_4num})'
    exp_4 = fr'(?P<date>{month_num}/{year_4num})'
    exp_5 = fr'(?P<date>{year_4num})'

    dates_1 = df.str.extract(exp_1, expand=False)
    dates_2 = df.str.extract(exp_2, expand=False)
    dates_3 = df.str.extract(exp_3, expand=False)
    dates_4 = df.str.extract(exp_4, expand=False)
    dates_5 = df.str.extract(exp_5, expand=False)

    dates =  pd.to_datetime(dates_1['date'].fillna(dates_2['date']).fillna(dates_3['date']).fillna(dates_4['date']).fillna(dates_5['date']))
    
    return dates.sort_values().index
#           
date_sorter()

Int64Index([  9,  84,   2,  53,  28, 474, 153,  13, 129,  98,
            ...
            152, 235, 464, 253, 427, 231, 141, 186, 161, 413],
           dtype='int64', length=500)